In [2]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [3]:
import transformers
from datasets import load_dataset, load_metric
from transformers import TrainingArguments

In [4]:
args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [5]:
import pandas as pd

In [6]:
df  = pd.read_csv("../data/data_pairs_2.csv").drop(['title'],axis=1)
df = df[100:]
df.to_csv("../data/data_pairs.csv",index=False)

In [7]:
medium_datasets = load_dataset("csv",data_files="../data/data_pairs.csv")
medium_datasets

Using custom data configuration default-4c3cb110bbfbeb90


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to C:\Users\zdahmani\.cache\huggingface\datasets\csv\default-4c3cb110bbfbeb90\0.0.0\652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['markdown', 'code'],
        num_rows: 7472
    })
})

In [8]:
datasets_train_test = medium_datasets["train"].train_test_split(test_size=750)
datasets_train_validation = datasets_train_test["train"].train_test_split(test_size=750)

medium_datasets["train"] = datasets_train_validation["train"]
medium_datasets["validation"] = datasets_train_validation["test"]
medium_datasets["test"] = datasets_train_test["test"]

In [9]:
medium_datasets["train"] = medium_datasets["train"].shuffle()
medium_datasets["validation"] = medium_datasets["validation"]
medium_datasets["test"] = medium_datasets["test"].shuffle()

In [10]:
import nltk
#nltk.download('punkt')
import string
from transformers import AutoTokenizer, PLBartForConditionalGeneration, PLBartTokenizer


tokenizer = PLBartTokenizer.from_pretrained("uclanlp/plbart-python-en_XX", src_lang="python", tgt_lang="en_XX")

In [11]:
medium_datasets_cleaned = medium_datasets.filter(
    lambda example: (len(example['markdown']) <= 512) and
    (len(example['code']) <= 512)
)

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [12]:
medium_datasets_cleaned

DatasetDict({
    train: Dataset({
        features: ['markdown', 'code'],
        num_rows: 5521
    })
    validation: Dataset({
        features: ['markdown', 'code'],
        num_rows: 689
    })
    test: Dataset({
        features: ['markdown', 'code'],
        num_rows: 686
    })
})

In [13]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zdahmani\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
prefix = "summarize: "
max_input_length = 512
max_target_length = 64

def clean_text(code):
  sentences = nltk.sent_tokenize(code.strip())
  sentences_cleaned = [s for sent in sentences for s in sent.split("\n")]

  text_cleaned = "\n".join(sentences_cleaned)
  return text_cleaned

def preprocess_data(examples):
  texts_cleaned = [clean_text(text) for text in examples["code"]]
  inputs = [prefix + code for code in texts_cleaned]
  model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

  # Setup the tokenizer for targets
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples["markdown"], max_length=max_target_length, 
                       truncation=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [15]:
tokenized_datasets = medium_datasets_cleaned.map(preprocess_data,batched=True)

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [16]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['markdown', 'code', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5521
    })
    validation: Dataset({
        features: ['markdown', 'code', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 689
    })
    test: Dataset({
        features: ['markdown', 'code', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 686
    })
})

In [17]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq,Seq2SeqTrainingArguments, Seq2SeqTrainer

In [18]:
batch_size = 8

model_name = "uclanlp/plbart-python-en_XX"
model_dir = "https://huggingface.co/uclanlp/plbart-python-en_XX"

args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="steps",
    eval_steps=100,
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="steps",
    save_steps=200,
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    report_to="tensorboard"
)

In [19]:
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [20]:
metric = load_metric("rouge")

In [21]:

import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) 
                      for label in decoded_labels]
    
    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [22]:
plbart_model = PLBartForConditionalGeneration.from_pretrained("uclanlp/plbart-python-en_XX")

In [23]:
def model_init():
    return PLBartForConditionalGeneration.from_pretrained("uclanlp/plbart-python-en_XX")

trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=Seq2SeqTrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch"),
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

loading configuration file https://huggingface.co/uclanlp/plbart-python-en_XX/resolve/main/config.json from cache at C:\Users\zdahmani/.cache\huggingface\transformers\68f40642f8534e3482166065ce817305c07e1f4b8ea96013fe62ab865088bddb.8929a51af95d04be1b1d966435fd1ad4a48aca2f2953bc4e4a354b1f1dfd0b55
Model config PLBartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": [
    "PLBartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "dropout": 0.1,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "max_position_embeddings": 1024,
  "model_type": "plbart",
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "scal

In [24]:
from huggingface_hub import list_repo_files

list_repo_files(repo_id= "uclanlp/plbart-python-en_XX")

['.gitattributes',
 'config.json',
 'pytorch_model.bin',
 'sentencepiece.bpe.model']

In [25]:
import torch
torch.cuda.empty_cache()
import gc

gc.collect()

torch.cuda.empty_cache()

In [26]:
trainer.train()

loading configuration file https://huggingface.co/uclanlp/plbart-python-en_XX/resolve/main/config.json from cache at C:\Users\zdahmani/.cache\huggingface\transformers\68f40642f8534e3482166065ce817305c07e1f4b8ea96013fe62ab865088bddb.8929a51af95d04be1b1d966435fd1ad4a48aca2f2953bc4e4a354b1f1dfd0b55
Model config PLBartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": [
    "PLBartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "dropout": 0.1,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "max_position_embeddings": 1024,
  "model_type": "plbart",
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "scal

  0%|          | 0/2073 [00:00<?, ?it/s]

Saving model checkpoint to test_trainer\checkpoint-500
Configuration saved in test_trainer\checkpoint-500\config.json


{'loss': 4.4928, 'learning_rate': 3.79401833092137e-05, 'epoch': 0.72}


Model weights saved in test_trainer\checkpoint-500\pytorch_model.bin
tokenizer config file saved in test_trainer\checkpoint-500\tokenizer_config.json
Special tokens file saved in test_trainer\checkpoint-500\special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `PLBartForConditionalGeneration.forward` and have been ignored: markdown, code. If markdown, code are not expected by `PLBartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 689
  Batch size = 8


  0%|          | 0/87 [00:00<?, ?it/s]

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'list'